In [10]:
import pandas as pd
import numpy as np

# Step 1: Load data input and initialize parameters

# Load data input
forecast_df = pd.read_csv('forecast_next6m_long.csv')
print(forecast_df.head(5))

#  Initialize Parameters
lead_time_month = 25 / 30  # Conver lead time into months (~0.83)
soh_target_ratio = 0.5 # Ending SOH = 0.5 * Next month's forecast


      SKU Forecast_Month  Forecasted_Sales
0  6HSD4J        2022-02     704638.750000
1  6HSD4J        2022-03     576718.083333
2  6HSD4J        2022-04     446847.161111
3  6HSD4J        2022-05     576067.998148
4  6HSD4J        2022-06     533211.080864


In [7]:
# Find the Ordering Plan for Top 10 SKUs

order_plan = []

# ---Step2: For each SKU:
for sku in forecast_df['SKU'].unique():
    sku_data = forecast_df[forecast_df['SKU'] == sku].reset_index(drop=True)
    forecast = sku_data['Forecasted_Sales'].values
    months = sku_data['Forecast_Month'].values

    # Set initial SOH (for each SKU)
    soh = soh_target_ratio * forecast[0]
    
    # ---Step3: For each month in the forecast horizon
    for i in range(6): # consider the next 6 moths
        # Target SOH at the end of month = next month's forecast × 0.5, OR = last month if at end
        if i < 5:
            target_soh_next = soh_target_ratio * forecast[i+1]
        else:
            target_soh_next = soh_target_ratio * forecast[i]

        # Sales forecast of month i
        sales = forecast[i]

        # Calculate order quantity so that ending SOH matches the target after sales
        # Order arrives after lead time; so we need to plan ahead. For months where the order arrives after the 6th period, just use the current period
        order_qty = max(target_soh_next + sales - soh, 0)

        order_plan.append({
            'SKU': sku,
            'Month': months[i],
            'Sales_Forecast': sales,
            'Order_Quantity': order_qty,
            'SOH_Begin': soh,
            'SOH_End': soh + order_qty - sales
        })

        # Update SOH for next period
        soh = soh + order_qty - sales


In [15]:
# Step 3: Store results in df and export results

forecast_order_plan = pd.DataFrame(order_plan)
forecast_order_plan.head()

,SKU,Month,Sales_Forecast,Order_Quantity,SOH_Begin,SOH_End
0,6HSD4J,2022-02,704638.750000,640678.416667,352319.375000,288359.041667
1,6HSD4J,2022-03,576718.083333,511782.622222,288359.041667,223423.580556
2,6HSD4J,2022-04,446847.161111,511457.579630,223423.580556,288033.999074
3,6HSD4J,2022-05,576067.998148,554639.539506,288033.999074,266605.540432
4,6HSD4J,2022-06,533211.080864,525959.913786,266605.540432,259354.373354


In [24]:
# Conver to wide  format for easy view
# Wide format: each SKU as a column, rows are Month, values are Order_Quantity

wide_order = forecast_order_plan.pivot(index='Month', columns='SKU', values=['Order_Quantity']).reset_index()

# To display in notebook without index
from IPython.display import display
display(wide_order.style.hide(axis='index')) 

# Export to CSv
# wide_order.to_csv('ordering_plan.csv', index=False)